In [78]:
import pandas as pd
import json 
from sentence_transformers import SentenceTransformer
import glob
import faiss
from fuzzywuzzy import fuzz

C:\Users\J C SINGLA\miniconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# !pip install pandas
# !pip install sentence-transformers
# !pip install openpyxl
# !pip install fuzzywuzzy

## Load model

In [4]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [5]:
def split_data(data):
    return data["raw_text"].split("\n")

In [6]:
def read_json_from_folders(directory_path):
    """
    Reads all JSON files from each folder in the specified directory.

    :param directory_path: Path to the directory containing folders of JSON files.
    :return: A list of dictionaries where each dictionary contains data from a single JSON file.
    """
    all_data = []
    filenames = []
    search_pattern = f"{directory_path}/*/*.json"
    for file_path in glob.glob(search_pattern):
        with open(file_path, 'r') as file:
            data = json.load(file)
            data = split_data(data)
            filenames.extend([file_path]*len(data))
            all_data.extend(data)
    return all_data, filenames

In [7]:
directory_path = r'C:\Users\J C SINGLA\Downloads\External - take_home_challenge_(withJSONs)\take_home_challenge_(withJSONs)\raw_text'
all_data, filenames = read_json_from_folders(directory_path)

In [8]:
doc_name = "DR--110685614"
subset_str = "C:\\Users\\J C SINGLA\\Downloads\\External - take_home_challenge_(withJSONs)\\take_home_challenge_(withJSONs)\\raw_text\\" +doc_name+"\\raw_text.json"

In [9]:
sentence_embeddings = model.encode(all_data)

In [10]:
sentence_embeddings.shape

(428, 768)

# Index Flat2

In [11]:
d = sentence_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.is_trained

In [12]:
index.add(sentence_embeddings)
index.ntotal

428

# Ground Truth

In [41]:
ground_truth_path = r"C:\Users\J C SINGLA\Downloads\External - take_home_challenge_(withJSONs)\take_home_challenge_(withJSONs)\document_questions.xlsx"

In [42]:
ground_truth = pd.read_excel(ground_truth_path)

In [43]:
ground_truth

,document,relevant questions,answer,complexity
0,DR--185549702_INTRO,"What is meant by ""computational finance""?",NaN,no raw_text
1,DR--185549702_INTRO,What is meant by 'investor heterogeneity'?,NaN,no raw_text
2,DR--185549702_INTRO,What was the revolution witnessed by finance i...,NaN,no raw_text
3,DR--185549702_INTRO,Why do you think financial markets are viewed ...,NaN,no raw_text
4,DR--185549702_INTRO,Why are the financial markets appealing applic...,NaN,no raw_text
5,DR--185549702_INTRO,What are the different design questions in con...,NaN,no raw_text
6,DR--185549702_INTRO,Discuss Lettau’s framework using various mathe...,NaN,no raw_text
7,DR--185549702_INTRO,The exchange rate is given by what formula?,NaN,no raw_text
8,DR--185549702_INTRO,Discuss the criticisms of agent-based markets.,NaN,no raw_text
9,DR--185549702_INTRO,How is timing an ignored final problem?,NaN,no raw_text


In [44]:
ground_truth_text = ground_truth[ground_truth["complexity"]=="text"].copy()

In [45]:
ground_truth_text

,document,relevant questions,answer,complexity
19,DR--182866691_INTRO,What is the Oregon Section's annual Traffic Bo...,The Student Liaison Committee had another succ...,text
20,DR--182866691_INTRO,The participating schools of the Traffic Bowl ...,This year we had 52 students from six universi...,text
21,DR--182866691_INTRO,How do you think the donations from the local ...,University of Portland took home the grand pri...,text
22,DR--182866691_INTRO,Discuss the Oregon Section's website.,This year the Oregon Section retained the doma...,text
23,DR--182866691_INTRO,What do you know about the 18th annual ITE Gol...,The Oregon Section hosted its 18th annual ITE ...,text
24,DR--182866691_INTRO,What are the standard rules of a golf game?,NaN,text
25,DR--182866691_INTRO,How do you think the token gifts motivate the ...,NaN,text
26,DR--182866691_INTRO,How was the District 6 meeting a huge success?,The District 6 meeting was a huge success in t...,text
27,DR--182866691_INTRO,Which award was granted over the past year? To...,Award Name: Traffic Bowl - First Place Award R...,text
28,DR--14627260_INTRO,How do you think sexual harassment as an issue...,NaN,text


# Inference

In [61]:
def test_case():
    query = ["What legislative suggestions pertaining to workplace sexual harassment does the firm provide in its submission?", 
                  "What legislative suggestions pertaining to workplace sexual harassment does the firm provide in its submission?"]
    return faiss_inference(query)[0] == faiss_inference(query)[1]

In [55]:
def faiss_inference(query, k=4):
    if isinstance(query, list):
        xq = model.encode(query)
        D, I = index.search(xq, k)
        retrieved_items = [[all_data[i] for i in sublist] for sublist in I]
        assert test_case()
    else:
        xq = model.encode([query])
        D, I = index.search(xq, k)
        retrieved_items = [all_data[i] for i in list(I[0])]
    return retrieved_items    

In [66]:
test_data = list(ground_truth_text["relevant questions"])

In [67]:
ret_context = faiss_inference(test_data)

# Testing 

In [158]:
def recall_k(test_labels, ret_context, k=len(ret_context[0])):
    ctr = 0
    correct, incorrect = [], []
    for i in range(len(ret_context)):
        done = False
        if isinstance(test_labels[i], float):
            # handle case for when answer is not present in the pdf
            continue
        for j in range(min(k,len(ret_context))):
            if fuzz.ratio(test_labels[i], ret_context[i][j])>=95:
                # print ("test_labels:", test_labels[i], "\n", "ret_context:", ret_context[i][j])
                ctr += 1
                done = True
                break
                
        if done == False:
            incorrect.append({test_labels[i]:ret_context[i]})

        else:
            correct.append({test_labels[i]:ret_context[i]})

    return ctr/len(ret_context), incorrect, correct

In [159]:
recall, incorrect, correct = recall_k(test_labels, ret_context)

In [164]:
recall

0.35714285714285715

In [211]:
incorrect[3].keys()

dict_keys(["Award Name: Traffic Bowl - First Place Award Recipient's Name: University of Portland Purpose of Award: To Encourage Student Participation in Oregon Section ITE Form of Award: $400 to Student Chapter"])

In [208]:
incorrect[0].values()

dict_values([["The Student Liaison Committee had another successful year in attracting schools from the Pacific Northwest to compete in the Oregon Section's annual Traffic Bowl Competition. The Traffic Bowl is a Jeopardy-based trivia contest where students must answer questions on a variety of traffic and transportation trivia. This year the competition was held on November 15, 2007 at McMenamin's Edgefield just east of Portland, Oregon. ", 'Donations:  $1,450 (Including Traffic Bowl prizes to student chapters) ', 'This year we had 52 students from six universities attending the event. Participating schools were: ', 'Approximately 13 transportation related bills were passed by the Oregon Legislative Assembly during the 2007 session. The topics of the bills vary widely, including issues such as expanding the use of photo radar and red light enforcement, various signing requirements, and gates for light rail pedestrian crossings. Details on the bills are provided to the Oregon Section me

In [ ]:
# missing chunking - 3
# not using tokens for search - 2
# too many references to fetch - 1

In [196]:
top10 = faiss_inference("In what cases should online sexual harassment be criminalized according to the submission?", 10)

In [206]:
top10[9]

'Seek injunctive relief and damages from the perpetrators of on-line sexual harassment, and '

# Better chunking

In [214]:
from llmsherpa.readers import LayoutPDFReader

In [215]:
# !pip install llmsherpa

In [243]:
os.getcwd()

'C:\\Users\\J C SINGLA\\Downloads\\External - take_home_challenge_(withJSONs)\\take_home_challenge_(withJSONs)'

In [216]:
llmsherpa_api_url = "https://readers.llmsherpa.com/api/document/developer/parseDocument?renderFormat=all"


In [248]:
pdf_url = "/Users/DR--182866691_INTRO.pdf" # also allowed is a file path e.g. /home/downloads/xyz.pdf


In [249]:
pdf_reader = LayoutPDFReader(llmsherpa_api_url)

In [250]:
doc = pdf_reader.read_pdf(pdf_url)

In [252]:
for chunk in doc.chunks():
    print (chunk.to_context_text(), "\n")
    

Oregon Section 2007/2008 Annual Report
INSTITUTE OF TRANSPORTATION ENGINEERS 

Oregon Section 2007/2008 Annual Report
OREGON SECTION Annual Report April 1, 2007 to March 31, 2008 

Oregon Section 2007/2008 Annual Report > Table of Contents
Section 

Oregon Section 2007/2008 Annual Report > Table of Contents
Page 

Oregon Section 2007/2008 Annual Report > Table of Contents > 3. Finances…………………………………………………………………………………………………
5 

Oregon Section 2007/2008 Annual Report > Table of Contents > 4. Legislative Activities……………………………………………………………………………………
6 

Oregon Section 2007/2008 Annual Report > 1. OFFICERS
 | President | Vice President | Secretary/Treasurer | Past President
 | --- | --- | --- | ---
 | Pamela O’Brien | Todd Mobley | Bikram Raghubansh | Carl Springer
 

Oregon Section 2007/2008 Annual Report > 1. OFFICERS > Officers and Committee Chairs
 | OFFICERS | 2006-2007 | 2007-2008
 | --- | --- | ---
 | President | Carl Springer | Pamela O’Brien
 | Vice President | Pamela O’Brien | Todd M